# Симплекс-метод 🤔

## Что такое задача ЛП

### Общий вид задачи ЛП

$$X=(x_1,x_2,\dots,x_n)^r \in R^n$$
_X_ - переменные
$$A=\begin{pmatrix}
 a_{1,1} & \cdots & a_{1,n} \\
\vdots & \ddots & \vdots \\
a_{m,1} & \cdots & a_{m,n} 
\end{pmatrix}=(A_1,\dots,A_n)$$
_A_ - коэфициенты
$$f(x)=cx=<c,x>=\sum^n_{j=1}c_jx_j\to extr$$
_f(x)_ куда-то стремится
$$\begin{cases}
\sum^n_{j=1}a_{ij}x_j<=b_i,i=1\dots r_1 \\
\sum^n_{j=1}a_{ij}x_j>=b_i,i=1+r_1\dots r_2 \\
\sum^n_{j=1}a_{ij}x_j=b_i,i=1+r_2\dots m 
\end{cases}$$
Есть какие-то ограничения
### Каноническая форма
$$f(x) \to extr \\
AX = b \\
X>=0, b>=0$$
Ограничения сводятся к равенствам, _иксы_ и правые части огранечений неотрицательны

__Любая задача сводится к канонической__

- Неравенства с отрицательными $b_i$ умножаем на (-1).
- Если неравенство вида ($<=$), то к левой части добавляем $s_i$ – добавочную переменную, и получаем равенство.
- Если неравенство вида ($>=$), то из левой части вычитаем $s_i$, и получаем равенство.
- Делаем замену переменных: \
Если $x_i ≤ 0$, то $x_i'= -x_i ≥ 0$ \
Если $x_i$ — любой, то $x_i= x_i' - x_i''$, где $x_i', x_i''≥ 0$

## Собственно симплекс-метод

### Что нужно

Для того, чтобы применить симлекс-метод для решения задачи, необходимо выполнение следующих условий:
- Задача должна иметь каноническую форму.
- У задачи должен быть базис.

Базис - множество таких $x_i$, что в соответствующих им столбцах матрицы $A_i$, находится одна $1$ и все остальные $0$. При этом кол-во $x_i$ - состовляющих базиса равно числу строк в матрице $A$, для каждой строки в этой матрице в базисе значение $1$ имеет только один $x_i$

### Как оно работает

Оптимальное решение в задачах ЛП всегда лежит на угловой точке. \
Симплекс метод позволяет эффективно найти решение задачи путём перемещения между этими точками.
Начиная решения с точки, соответствующей начальному базису, в ходе решения задачи изменяется базис (т.е. происходит перемещение между угловыми точками).

Алгоритм:

- Находим переменную, которую будем вводить в базис:
    
    - Находим оценку для каждой переменной.
    - При решении задачи на максимизацию, вводиться будет переменная с максимальной положительной оценкой. \
    При решении задачи на минимизацию, вводиться будет переменная с минимальной отрицательной оценкой. 
    
    Здесь в базис вводится переменная, изменение которой _лучше_ всего влияет на измененние функции.

- Находим переменную, которую будем убирать из базиса:
    - Столбец правых частей $b$ почленно делим на соответствующий вводимому элементу ($x_i$), столбец $A_i$.
    - Выбираем минимальный положительный элемент.

- Переходим к новому базису:
    - Используем метод Жордана-Гаусса и получаем новый базис, новый вектор правых частей $b$, и новые коэфициенты $A$.

- Проверяем оценки:
    - Если среди оценок для новых данных есть позволяющие перейти к следующей итерации - переходим, иначе - решение получено.

### Как мы его заставили работать

Т.к. нет ничего интересного в том, чтобы разбирать строку с данными, приводить задачу к каноническому виду, рассматривать случаи максимизации и минимизации (реализован только первый) или проверять данные на соответствие условью (_если есть ошибка - данные неверны?_), то мы будет взаимодействовать с изначально подготовленными данными:
- Функция для оптимизации - _набора_ коэфициентов при переменных.
- Коэфициентами в ограничивающих функциях - опять же _набор наборов_ коэфициентов, но двумерный.
- Правая часть ограничений - _набор_ чисел.
- Базис - _набор_ битовых значений, размерность которого соответствует размерности _набора_ коэфициентов функции:
    - 0 - переменная не принадлежит к базису.
    - 1 - переменная принадлежит к базису.
 
Далее благодоря набору функций, работаем с данными, меняем базисы пока нас всё не устраивает...

 _Известно, что так делать не хорошо, но _набор_ - обычный питоновский список, оптимизация всё-таки не важна._ \
 _P.S. прошу прощения за всё, что написано выше, наверное можно было и лучше объяснить_ \
 _P.P.S Также заранее прошу прощения за всё, что написано ниже._ 

In [2]:
# Данные
function = [4, 5, 4, 0, 0, 0]

factors = [[2, 3, 6, 1, 0, 0],
           [4, 2, 4, 0, 1, 0],
           [4, 6, 8, 0, 0, 1]]

basis = [0, 0, 0, 1, 1, 1]

solutions = [240, 200, 160]


In [3]:
from termcolor import colored

# Находим те индексы которые относятся к базису ([0, 0, 1, 1, 1] -> [2, 3, 4])
def calc_basis_indexes(basis: list) -> list:
    return list(filter(lambda x: x != -1, map(lambda x: x if basis[x] else -1, range(len(basis)))))


# Считаем оценку, позволяющую понять, что следует ввести в базис или найдено ли оптимальное решение
def calculate_costs(function, basis, factors):
    costs = []
    basis_indexes = calc_basis_indexes(basis)
    for i in range(len(function)):
        subtotal = -function[i]
        for j in range(len(basis_indexes)):
            subtotal += function[basis_indexes[j]] * factors[j][i]
        costs.append(subtotal)
    return costs
# Дополняем наш вектор решений нулями
def extend_solutions(solutions, basis):
    res = list(solutions)
    for i in range(len(basis)):
        if basis[i] == 0:
            res.insert(i, 0)
    return res


# Тут должно быть очевидно
def dot_product(op1, op2):
    return [op1[i] * op2[i] for i in range(len(op1))]


# Какой-никакой, но форматированный вывод
def print_table(function, factors, basis, solutions, end=False):
    func = [f"{i:^11.2f}" for i in function]
    fact = [[f"{j:^11.2f}" for j in i] for i in factors]
    sols = extend_solutions(solutions, basis)
    sols.append(sum(dot_product(sols, function)))
    sols = [f"{i:^11.2f}" for i in sols]
    basis_indexes = calc_basis_indexes(basis)
    bas = [f"|  x{i+1:<3d} " for i in basis_indexes]
    separator = "+-------"+"+-----------"*(len(func)+1)+"+"
    costs = [f"{i:^11.2f}" for i in calculate_costs(
        function, basis, factors)]
    print(separator)
    print("|   f   =" + "+".join(func) + "=" +
          colored(sols[-1], "green" if end else 'red')+"|")
    print("|  x =  (" + ",".join(sols[:-1]) + ")" + "           |")
    print(separator)
    print("|      ", *
          [f"|    x{i+1:<5d}" for i in range(len(func))], "|           |")
    for i in range(len(fact)):
        print(bas[i], *fact[i], sols[basis_indexes[i]], sep="|", end="|\n")
    print(separator)
    print("|   Δ   ", *costs, "           ", sep="|", end="|\n")
    print(separator)

print_table(function, factors, basis, solutions)

+-------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+
|   f   =   4.00    +   5.00    +   4.00    +   0.00    +   0.00    +   0.00    =   0.00    |
|  x =  (   0.00    ,   0.00    ,   0.00    ,  240.00   ,  200.00   ,  160.00   )           |
+-------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+
|       |    x1     |    x2     |    x3     |    x4     |    x5     |    x6     |           |
|  x4   |   2.00    |   3.00    |   6.00    |   1.00    |   0.00    |   0.00    |  240.00   |
|  x5   |   4.00    |   2.00    |   4.00    |   0.00    |   1.00    |   0.00    |  200.00   |
|  x6   |   4.00    |   6.00    |   8.00    |   0.00    |   0.00    |   1.00    |  160.00   |
+-------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+
|   Δ   |   -4.00   |   -5.00   |   -4.00   |   0.00    |   0.00    |   0.00    |           |
+-------+-----------+-----------+-----------+-----------+---

In [12]:
calculate_costs(function, basis, factors)

[-4, -5, -4, 0, 0, 0]

In [13]:
# Находим, что добавить
def index_of_min(in_):
    return in_.index(min(in_))

index_to_add = index_of_min(calculate_costs(function, basis, factors))
index_to_add

1

In [14]:
# Ищем какой элемент следует убрать из базиса
def find_index_to_remove(index_to_add: int, factors: list, solutions: list) -> int:
    min_value = 0
    min_index = 0
    while not min_value:
        if factors[min_index][index_to_add] > 0:
            min_value = solutions[min_index] / factors[min_index][index_to_add]
        min_index += 1
    min_index -= 1
    for i in range(min_index, len(solutions)):
        if factors[i][index_to_add]:
            value = solutions[i] / factors[i][index_to_add]
            if value < min_value and value > 0:
                min_value = value
                min_index = i
    return min_index

index_to_remove = find_index_to_remove(index_to_add, factors, solutions)
index_to_remove

2

In [15]:
# Меняем базис
def change_basis(index_to_add, index_to_remove, basis, factors, solutions):
    new_factors = []
    new_solutions = []
    basis[calc_basis_indexes(basis)[index_to_remove]] = 0
    basis[index_to_add] = 1
    new_index_position = basis[:index_to_add].count(1)
    solutions[index_to_remove] /= factors[index_to_remove][index_to_add]
    factors[index_to_remove] = [i / factors[index_to_remove][index_to_add]
                                for i in factors[index_to_remove]]
    for i in range(len(factors)):
        if i != index_to_remove:
            new_factors.append([factors[i][j] - factors[index_to_remove]
                               [j] * factors[i][index_to_add] for j in range(len(factors[i]))])
            new_solutions.append(solutions[i] - solutions[index_to_remove]
                                 * factors[i][index_to_add])
    new_solutions.insert(new_index_position, solutions[index_to_remove])
    new_factors.insert(new_index_position, factors[index_to_remove])
    return new_factors, new_solutions, basis

factors, solutions, basis = change_basis(index_to_add, index_to_remove, basis, factors, solutions)

In [16]:
# И далее - в цикле
while any(map(lambda x: x < 0, calculate_costs(function, basis, factors))):
    print_table(function, factors, basis, solutions)
    index_to_add = index_of_min(calculate_costs(function, basis, factors))
    index_to_remove = find_index_to_remove(index_to_add, factors, solutions)
    factors, solutions, basis = change_basis(index_to_add, index_to_remove, basis, factors, solutions)
print_table(function, factors, basis, solutions, True)

+-------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+
|   f   =   4.00    +   5.00    +   4.00    +   0.00    +   0.00    +   0.00    =  133.33   |
|  x =  (   0.00    ,   26.67   ,   0.00    ,  160.00   ,  146.67   ,   0.00    )           |
+-------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+
|       |    x1     |    x2     |    x3     |    x4     |    x5     |    x6     |           |
|  x2   |   0.67    |   1.00    |   1.33    |   0.00    |   0.00    |   0.17    |   26.67   |
|  x4   |   0.00    |   0.00    |   2.00    |   1.00    |   0.00    |   -0.50   |  160.00   |
|  x5   |   2.67    |   0.00    |   1.33    |   0.00    |   1.00    |   -0.33   |  146.67   |
+-------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+
|   Δ   |   -0.67   |   0.00    |   2.67    |   0.00    |   0.00    |   0.83    |           |
+-------+-----------+-----------+-----------+-----------+---

_вуаля_